<a href="https://colab.research.google.com/github/yunn333/AI/blob/main/%E5%B0%8D%E8%A9%B1%E6%A9%9F%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1.引入老師的基礎四套件

In [108]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### 2. 安裝並執行 Ollama

先到官網找到安裝程式並安裝。

In [109]:
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


###3.啟用ollama伺服器
在伺服器上拉一個
gemma3:1b

In [110]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [111]:
!ollama pull gemma3:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 7cd4618c1faf... 100% ▕▏ 815 MB                         
pulling e0a42594d802... 100% ▕▏  358 B                         
pulling dd084c7d92a3... 100% ▕▏ 8.4 KB                         
pulling 3116c5225075... 100% ▕▏   77 B                         
pulling 120007c81bf8... 100% ▕▏  492 B                         
verifying sha256 digest 
writing manifest 
success 


### 4. 用 OpenAI API 使用

導入open AI、API金鑰及API網址

In [112]:
import openai
from openai import OpenAI

In [113]:
api_key = "ollama"

In [114]:
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

### 5. 使用Gradio創造聊天機器人網頁
設定自己的聊天機器人與頁面，並開始應用

In [115]:
!pip install gradio

In [122]:
import gradio as gr

In [123]:
title = "22世紀機器人"
description = "你好，我是你的22世紀機器人,就跟多拉欸夢一樣，有問題的話都可以找我"
system = "你是一個很熱心的機器人，回應都需要應用到高科技的技術來解決使用者的問題，請用台灣習慣的中文來回應。"
description = "你好，我是你的22世紀機器人,有問題的話都可以找我"
model = "gemma3:1b"

In [124]:
messages = [{"role":"system", "content":system}]
initial_messages = [{"role":"system",
             "content":system},
            {"role":"assistant",
            'content':description}]

In [125]:
state = gr.State(messages)

In [126]:
def pipi(prompt, messages):
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    reply = chat_completion.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    #history = history + [[prompt, reply]]
    return messages, messages

In [127]:
chatbot = gr.Chatbot(type="messages")

In [ ]:
with gr.Blocks(title=title) as demo:
    gr.Markdown(f"## 🤖 {title}\n{description}")
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox(label="輸入訊息")
    state = gr.State(initial_messages.copy())  # 務必用 copy()

    msg.submit(fn=pipi, inputs=[msg, state], outputs=[chatbot, state])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://981696c0fd4f5e7f27.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
